# Web scraping KIDFamMap:

## Inhibitor csv File Generation by Web Scraping and Parsing KIDFamMap Data

Here we define and run a spider to extract the inhibitor data and UniProt IDs for each of the kinases that have inhibitor data available.

Once the data has been extracted from the website, it is cleaned and inserted into a Pandas data frame.

After that, the following steps take place:
- Remove commas from inhibitor names, to avoid problems when making the database from comma-separated files
- Translate kinase names to match "Entry name" (UniProt ID) in kinase table.
- Using information in the data frame, we generate a column of URLs that our web app can use to display images of the inhibitors' chemical structures.
- Convert kinases and inhibitors to uppercase.
- Remove duplicate rows.
- Remove unnecessary columns.
- Use this data frame to make two data frames: one listing the kinase-inhibitor pairs, and one listing each inhibitor alongside all of its information. These two data sets can then be linked via the inhibitor name in our relational database.
- Make primary key columns.
- Export as csv files.

Import packages

In [ ]:
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

Make a list of kinases names by simply copying and pasting them from the KIDFamMap website

In [ ]:
kinases = ['LATS1','LATS2','MASTL','RPS6KB1','RPS6KB2','RPS6KC1','RPS6KL1','BRSK1','BRSK2','HUNK','KIAA0999','MAPKAPK5','MARK1','MARK2','MARK3','MARK4','MGC42105','MKNK1','NUAK1','NUAK2','PHKG1','PHKG2','SIK1','SIK2','SNRK','STK40','TRIB1','TRIB2','TRIB3','TSSK1B','TSSK2','TSSK3','TSSK4','TSSK6','TTN','TTBK1','TTBK2','VRK2','VRK3','CK2a1-rs','CDK4','MAPK13','MAPK4','MAPK6','SgK424','AAK1','BMP2K','BUB1B','C9orf96','CDC7','CHUK','DKFZp761P0423','DSTYK','EIF2AK1','EIF2AK3','EIF2AK4','FLJ23356','GAK','IKBKB','IKBKE','KIAA2002','LOC646643','LOC91461','MGC16169','MOS','NRBP1','NRBP2','PBK','PDIK1L','PIK3R4','PINK1','PKMYT1','PXK','RNASEL','SBK1','SCYL1','SCYL2','SCYL3','SGK110','STK31','STK35','TBK1','TEX14','TLK1','TLK2','TP53RK','UHMK1','ULK1','ULK4','WNK1','WNK2','WNK3','WNK4','STLK6-rs','MAP2K7','MAP3K1','MAP3K14','MAP3K15','MAP3K2','MAP3K3','MAP3K4','MAP3K5','MAP3K6','MAP3K8','MAP4K1','MAP4K2','PAK6','STK3','STK4','TAOK1','TAOK2','TAOK3','YSK4','EPHA4','EPHA5','EPHA6','EPHA8','FLT3','LMTK2','ACVR2B','ANKK1','IRAK2','IRAK3','LIMK1','LIMK2','LRRK1','LRRK2','MLKL','RIPK1','RIPK2','RIPK3','RIPK4','TESK1','TESK2','TNNI3K','ADCK1','ADCK2','ADCK4','ADCK5','ALPK1','ALPK2','ALPK3','ATM','ATR','BCKDK','CABC1','EEF2K','FRAP1','PDK1','PDK2','PDK3','PDK4','PRKDC','RIOK1','RIOK2','RIOK3','SMG1','TRPM6','TRPM7','TRRAP','GUCY2C','GUCY2D','GUCY2F','NPR1','NPR2','BCR','BRD2','BRD3','BRD4','BRDT','FASTK','HSPB8','STK19','TAF1','TAF1L','TRIM24','TRIM28','TRIM3','ADRBK2','AKT3','CDC42BPA','CDC42BPB','CDC42BPG','CIT','FLJ25006','GRK1','GRK4','GRK5','GRK7','LOC375449','MAST1','MAST2','MAST3','PKN1','PKN2','PKN3','PRKACB','PRKACG','PRKCD','PRKCE','PRKCG','PRKCH','PRKCZ','PRKG1','PRKG2','PRKX','PRKY','ROCK2','RPS6KA2','RPS6KA3','RPS6KA4','RPS6KA6','SGK2','SGK3','STK32A','STK32B','STK32C','STK38','STK38L','CAMK1','CAMKV','DCLK1','DCLK2','DCLK3','KALRN','LOC729871','MELK','MYLK','MYLK2','MYLK3','OBSCN','PIM3','PNCK','PRKAA1','PRKD1','PRKD2','PRKD3','PSKH1','PSKH2','STK17A','STK33','TRIO','CSNK1A1','CSNK1A1L','CSNK1D','CSNK1E','CCRK','CDC2','CDC2L5','CDC2L6','CDK10','CDK11A','CDK15','CDK3','CDK8','CDKL1','CDKL2','CDKL3','CDKL4','CDKL5','CLK4','CRKRS','DYRK1B','DYRK3','DYRK4','GSK3A','HIPK1','HIPK2','HIPK3','HIPK4','ICK','MAK','MAPK15','MAPK7','NLK','PCTK2','PCTK3','PFTK1','PRPF4B','RAGE','SRPK3','AURKB','AURKC','CAMKK1','ERN2','NEK1','NEK10','NEK11','NEK3','NEK4','NEK5','NEK6','NEK8','NEK9','PLK2','PLK3','STK36','ULK2','ULK3','WEE2','ALS2CR2','MAP2K3','MAP2K5','MAP4K3','MAP4K4','MAP4K5','MINK1','MYO3A','MYO3B','NRK','PAK2','PAK3','STK39','AATK','AXL','BLK','BMX','DDR1','DDR2','EPHA1','EPHA10','EPHB1','EPHB2','EPHB3','EPHB6','FER','FGFR3','FGFR4','FGR','FLT4','FRK','INSRR','LMTK3','LTK','MATK','MUSK','NTRK1','NTRK2','NTRK3','PDGFRA','PDGFRB','PTK6','PTK7','ROR1','ROR2','ROS1','RYK','SRMS','STYK1','TEC','TIE1','TNK1','TXK','TYRO3','YES1','ACVR1B','ACVR1C','AMHR2','ARAF','BMPR1A','IRAK1','KIAA1804','KSR1','MAP3K10','MAP3K11','MAP3K12','MAP3K13','TGFBR2','ZAK','ADRBK1','AKT1','AKT2','DMPK','GRK6','PDPK1','PRKACA','PRKCA','PRKCB','PRKCI','PRKCQ','ROCK1','RPS6KA1','RPS6KA5','SGK1','CAMK1D','CAMK1G','CAMK2A','CAMK2B','CAMK2D','CAMK2G','CAMK4','CASK','CHEK1','CHEK2','DAPK1','DAPK2','DAPK3','MAPKAPK2','MAPKAPK3','MKNK2','MYLK4','PASK','PIM1','PIM2','PRKAA2','STK11','STK17B','CSNK1G1','CSNK1G2','CSNK1G3','VRK1','CDK2','CDK5','CDK6','CDK7','CDK9','CLK1','CLK2','CLK3','CSNK2A1','CSNK2A2','DYRK1A','DYRK2','GSK3B','MAPK1','MAPK10','MAPK11','MAPK12','MAPK14','MAPK3','MAPK8','MAPK9','PCTK1','SRPK1','SRPK2','AURKA','BUB1','CAMKK2','EIF2AK2','ERN1','GSG2','NEK2','NEK7','PLK1','PLK4','STK16','TTK','WEE1','MAP2K1','MAP2K2','MAP2K4','MAP2K6','MST4','OXSR1','PAK1','PAK4','PAK7','SLK','STK10','STK24','STK25','STRADA','TNIK','ABL1','ABL2','ALK','BTK','CSF1R','CSK','EGFR','EPHA2','EPHA3','EPHA7','EPHB4','ERBB2','ERBB3','ERBB4','FES','FGFR1','FGFR2','FLT1','FYN','HCK','IGF1R','INSR','ITK','JAK1','JAK2','JAK3','KDR','KIT','LCK','LYN','MERTK','MET','MST1R','PTK2','PTK2B','RET','SRC','SYK','TEK','TNK2','TYK2','ZAP70','ACVR1','ACVR2A','ACVRL1','BMPR1B','BMPR2','BRAF','ILK','IRAK4','KSR2','MAP3K7','MAP3K9','RAF1','TGFBR1']

Make URLs from the kinases list

In [ ]:
kinase_urls = []

for i in kinases:
    url = "http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=" + str( i ) + "&Query_Pid=" + str( i )
    kinase_urls.append( url )

Make a spider

In [ ]:
# Use the scrapy.Spider class to make an inhibitor-details-scraping 
# spider

class InhibitorSpider( scrapy.Spider ):
    
    name = "inhibitor_spider"
    
    # Define the first action to take
        
    def start_requests( self ): 
        
        # Go to the website at the above URLs and get a response object
        # which contains the HTML code for that web page
        # Define what to do with the response object
        # i.e. send it to the parse method defined below
        
        for url in kinase_urls:  
            yield scrapy.Request( url = url, callback = self.parse )      
 
    def parse( self, response ):
        
        # Each inhibitor list web page has information we'd like to
        # extract and place into a list "inhibs" 
        # (which we must initialise in the next cell rather than here)
        # We also want to extract the UniProt ID and kinase ID
        # in order to translate the latter to the former
        # for cross-referencing in our database
        # Define CSS or xpath locators to point to the required data 
        # and extract the text
       
        raw = response.css( 'div.result tbody > tr' ).extract()
        uniprot = response.xpath( '//a[@target="_blank"]/text()' ).extract()
        kin = response.css( 'td.tdleft5::text' ).extract()
        
        # Save the data in a list "fields"
        
        fields = [ field for field in raw ]
        
        # For each kinase, append the information about its
        # inhibitors to "inhibs"
        
        inhibs.append( fields )
        kins.append( kin )
        uniprots.append( uniprot )

Run the spider: crawl KIDFamMap for inhibitors

In [ ]:
inhibs = []
kins = []
uniprots = []

# Run the spider

process = CrawlerProcess()
process.crawl( InhibitorSpider )
process.start()

# N.B. kernel needs to be cleared before repeating

Clean up the data in "inhibs" and store in "inhibitors"

In [ ]:
inhibitors = []

for i in inhibs: # for each kinase
    for j in i: # for each kinase-inhibitor relationship
        chemical = []
        inh = j.split( "</td>" ) # Split row into individual fields
        for k in inh: # for each field, remove unnecessary characters
            field = k.replace( "<tr>", "" ) 
            field = field.replace( "\r", "" )
            field = field.replace( "\t", "" )
            field = field.replace( "<td>", "" )
            field = field.replace( "\n", "" )
            chemical.append( field ) # Make a row of cleaned, separate fields
        inhibitors.append( chemical ) # Add this row to "inhibitors"

Define column names, based on those on the KIDFamMap website

In [ ]:
headers = [ "Index", "Kinase", "Inhibitor", "Partial_Img_URL",
           "Ki_nM", "IC50_nM", "Kd_nM", "EC50_nM", "POC", "Source",
           "Link", "To_Remove" ]

Save the inhibitors information in a Pandas data frame

In [ ]:
inhibitors_df = pd.DataFrame( inhibitors, columns = headers )

Remove commas from inhibitors, to avoid problems in database

In [ ]:
nocomm = []

for i in inhibitors_df.Inhibitor:
    if "," in i:
        nocomm.append( i.replace( ",", "" ) )
    else:
        nocomm.append( i )

nocomm = pd.Series( nocomm )
     
inhibitors_df = inhibitors_df.assign( Inhibitor = nocomm )

Earlier we scraped "kins" and UniProt IDs "uniprots". Clean these up and generate a dictionary of kinase names with their UniProt IDs.

In [ ]:
kinases = []

for i in kins:
    kin = str( i )
    kin = kin.replace( "[", "" )
    kin = kin.replace( "]", "" )
    kin = kin.replace( "\\r", "" )
    kin = kin.replace( "\\t", "" )
    kin = kin.replace( "<td>", "" )
    kin = kin.replace( "\\n", "" )
    kin = kin.replace( "(', ')", "" )
    kin = kin.replace( "'", "" )
    kin = kin.replace( " ", "" )
    kin = kin.split( "," )
    kinases.append( kin[ 0 ] ) 

uniprot_ids = []

for i in uniprots:
    up = str( i )
    up = up.replace( "[", "" )
    up = up.replace( "]", "" )
    up = up.replace( "\\r", "" )
    up = up.replace( "\\t", "" )
    up = up.replace( "<td>", "" )
    up = up.replace( "\\n", "" )
    up = up.replace( "(', ')", "" )
    up = up.replace( "'", "" )
    up = up.replace( " ", "" )
    up = up.split( "," )
    uniprot_ids.append( str( up[ 0 ] ) )

In [ ]:
uniprot_dict = dict( list( zip( kinases, uniprot_ids ) ) )

In the inhibitors table (which so far only contains kinase names, not UniProt IDs), translate the kinase IDs to UniProt IDs and store in column

In [ ]:
UniProt_ID = []

for n, i in enumerate( inhibitors_df.Kinase ):
    if i in uniprot_dict.keys():
        uni = uniprot_dict.get( i )
        UniProt_ID.append( uni )
    else:
        UniProt_ID.append( float( 'NaN' ) )

UniProt_ID = pd.Series( UniProt_ID )
inhibitors_df = inhibitors_df.assign( UniProt_ID = UniProt_ID )

Using information in the data frame, generate a column of URLs for the inhibitors' chemical structure images. Our web app can subsequently use these to display images.

In [ ]:
IMG_URL = []

for n,i in enumerate( inhibitors_df.Partial_Img_URL ):
    URL = 'http://gemdock.life.nctu.edu.tw/kidfammap/data/png/'
    URL += str( inhibitors_df.Source[ n ] ) + "/"
    URL += str( i ) + ".png"
    IMG_URL.append( URL )

IMG_URL = pd.Series( IMG_URL )
inhibitors_df = inhibitors_df.assign( IMG_URL = IMG_URL )

Make "kinase" and "inhibitor" entries uppercase.

In [ ]:
uppercase_kinase = []
uppercase_inhib = []

for n,i in enumerate( inhibitors_df.Kinase ):
    uppercase_kinase.append( str( i ).upper() )
    uppercase_inhib.append( inhibitors_df.Inhibitor[ n ].upper() )

uppercase_kinase = pd.Series( uppercase_kinase )
uppercase_inhib = pd.Series( uppercase_inhib )

inhibitors_df = inhibitors_df.assign( Kinase = uppercase_kinase )
inhibitors_df = inhibitors_df.assign( Inhibitor = uppercase_inhib )

Make a temporary column combining the inhibitor and kinase names, to check for duplicates

In [ ]:
unique = []

for n,i in enumerate( inhibitors_df.Inhibitor ):
    uniq = str( i ) + "_" + str( inhibitors_df.Kinase[ n ] )
    unique.append( uniq )

unique = pd.Series( unique )
inhibitors_df = inhibitors_df.assign( UNIQUE = unique )

Drop any duplicate kinase-inhibitor pairs and reset the indices

In [ ]:
inhibitors_df = inhibitors_df.drop_duplicates( subset = "UNIQUE" )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Drop any rows without a UniProt ID

In [ ]:
inhibitors_df = inhibitors_df.dropna( subset = [ "UniProt_ID" ] )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Remove any columns not required for the web app

In [ ]:
inhibitors_df = inhibitors_df.drop( [ "Index", "To_Remove", "Partial_Img_URL", "Link", "UNIQUE" ], axis = 1 )

Make data frame of kinase-inhibitor pairs

In [ ]:
inhib_kin_df = inhibitors_df[ [ 'Kinase', 'Inhibitor', 'UniProt_ID' ] ]

Make data frame of inhibitors

In [ ]:
inhibitors_df = inhibitors_df.drop_duplicates( subset = "Inhibitor" )
inhibitors_df = inhibitors_df.drop( [ "Kinase" ], axis = 1 )
inhibitors_df = inhibitors_df.drop( [ "UniProt_ID" ], axis = 1 )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Make a column of primary keys in each data frame

In [ ]:
prim_key = []

count = 1

for i in inhibitors_df.Inhibitor:
    key = "IN" + "{:07d}".format( count )
    prim_key.append( key )
    count += 1

prim_key = pd.Series( prim_key )

inhibitors_df = inhibitors_df.assign( ID_IN = prim_key )

In [ ]:
prim_key = []

count = 1

for i in inhib_kin_df.Inhibitor:
    key = "KI" + "{:07d}".format( count )
    prim_key.append( key )
    count += 1

prim_key = pd.Series( prim_key )

inhib_kin_df = inhib_kin_df.assign( ID_KI = prim_key )

Write to csv

In [ ]:
inhib_kin_df.to_csv( "inhib_kin.csv", index = False )

In [ ]:
inhibitors_df.to_csv( "inhibitors.csv", index = False )